In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2

class CountrySalary:
    def __init__(self, country, monthly_salary):
        self.country = country
        self.monthly_salary = monthly_salary
    
    def __repr__(self):
        return f"Country: {self.country}, Monthly Salary: {self.monthly_salary}"

def process_salary(salary_text):
    if salary_text:
        salary_text = salary_text.replace('$', '').replace(',', '')
        try:
            salary = float(salary_text)  
            return salary
        except ValueError:
            return None  
    return None  
def get_country_salaries(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table', {'class': 'wikitable'})

    country_salaries = []

    
    for row in table.find_all('tr')[1:]: 
        cells = row.find_all('td')
        if len(cells) > 1:
            country = cells[0].get_text(strip=True).replace('*', '') 
            salary_text = cells[2].get_text(strip=True)
            salary = process_salary(salary_text)
            country_salaries.append(CountrySalary(country, salary))

    return country_salaries



In [ ]:




def save_to_postgresql(country_salaries):
    connection =  connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8" 
    )
    cursor = connection.cursor()

    for country_salary in country_salaries:
        print(country_salary.country)
        cursor.execute("SELECT id FROM public.country WHERE upper(country_name) = upper(%s)", (country_salary.country,))
        resultado = cursor.fetchone()

        if resultado:
            pais_id = resultado[0]
            print(pais_id)
            cursor.execute("""
            INSERT INTO salary.country_salary (m_salary, country_id)
            VALUES (%s, %s);
            """, (country_salary.monthly_salary, pais_id))
            connection.commit()

        
    cursor.close()
    connection.close()
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage'
country_salaries = get_country_salaries(url)

save_to_postgresql(country_salaries)

